In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline, Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
#from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.metrics import accuracy_score, precision_score, make_scorer

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn import metrics


import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def reading_dataframe(path, sep=None):

  """ Leitura de um arquivo csv"""

  dataframe = pd.read_csv(path, sep=sep)
  return dataframe


def column_empty(dataframe):

  """ Verifica as colunas que possuem dados vazios"""

  for col in (dataframe):
    if(dataframe[col].isnull().any()):
        print(col)

def values_estimate():
  return IterativeImputer()

In [4]:
df_credit_train= reading_dataframe("/content/drive/MyDrive/curso-lets-code/machine-learning_I/data/application_train.csv")

df_credit_test = reading_dataframe("/content/drive/MyDrive/curso-lets-code/machine-learning_I/data/application_test_student.csv")

In [5]:
df_credit_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


**Critério de seleção de variáveis**

 -  8 Maiores correlações positivas
 -  8 Menores correlações negativas

In [6]:
corr_sorting = df_credit_train.corr()['TARGET'].sort_values()

In [7]:
corr_sorting.head(n=8)

EXT_SOURCE_3      -0.179246
EXT_SOURCE_2      -0.160978
EXT_SOURCE_1      -0.158619
DAYS_EMPLOYED     -0.044862
FLOORSMAX_AVG     -0.043720
FLOORSMAX_MEDI    -0.043524
FLOORSMAX_MODE    -0.042958
AMT_GOODS_PRICE   -0.040060
Name: TARGET, dtype: float64

In [8]:
corr_sorting.tail(n=8)

FLAG_EMP_PHONE                 0.045894
REG_CITY_NOT_WORK_CITY         0.050255
DAYS_ID_PUBLISH                0.051572
DAYS_LAST_PHONE_CHANGE         0.054471
REGION_RATING_CLIENT           0.059170
REGION_RATING_CLIENT_W_CITY    0.061518
DAYS_BIRTH                     0.079414
TARGET                         1.000000
Name: TARGET, dtype: float64

In [9]:
credit_train_8 = df_credit_train[['EXT_SOURCE_3', 'EXT_SOURCE_2','EXT_SOURCE_1','DAYS_EMPLOYED',
                                  'FLOORSMAX_AVG','DAYS_LAST_PHONE_CHANGE','REGION_RATING_CLIENT','FLAG_EMP_PHONE','DAYS_BIRTH','TARGET']]

In [10]:
credit_train_8.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG,DAYS_LAST_PHONE_CHANGE,REGION_RATING_CLIENT,FLAG_EMP_PHONE,DAYS_BIRTH,TARGET
0,0.171468,0.699373,NaN,-813,0.1667,-589.0,2,1,-8676,0
1,0.768808,0.199651,0.541385,-223,0.0417,-1409.0,2,1,-13583,0
2,0.626304,0.704880,NaN,-6202,NaN,-675.0,2,1,-13993,0
3,0.810618,0.724576,NaN,365243,NaN,-1588.0,1,0,-22425,0
4,0.331251,0.706310,0.592466,-2763,0.1667,0.0,2,1,-18839,0


In [11]:
column_empty(credit_train_8)

EXT_SOURCE_3
EXT_SOURCE_2
EXT_SOURCE_1
FLOORSMAX_AVG
DAYS_LAST_PHONE_CHANGE


In [12]:
credit_train_8.isnull().mean().round(4)*100

EXT_SOURCE_3              19.81
EXT_SOURCE_2               0.22
EXT_SOURCE_1              56.42
DAYS_EMPLOYED              0.00
FLOORSMAX_AVG             49.79
DAYS_LAST_PHONE_CHANGE     0.00
REGION_RATING_CLIENT       0.00
FLAG_EMP_PHONE             0.00
DAYS_BIRTH                 0.00
TARGET                     0.00
dtype: float64

**Transformação dos Dados**
- Aplicação do método SimpleImputer para tratar os valores vazios.

In [ ]:
# imputer = SimpleImputer(strategy='median', missing_values=np.nan)

# imputer = imputer.fit(credit_train_8[['EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1', 'FLOORSMAX_AVG']])
# credit_train_8[['EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1', 'FLOORSMAX_AVG']] = imputer.transform(credit_train_8[['EXT_SOURCE_3','EXT_SOURCE_2','EXT_SOURCE_1', 'FLOORSMAX_AVG']])


In [ ]:
# credit_train_8.isnull().mean().round(4)*100

In [13]:
credit_train_8 = credit_train_8.fillna(0)
credit_train_8.head()

,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG,DAYS_LAST_PHONE_CHANGE,REGION_RATING_CLIENT,FLAG_EMP_PHONE,DAYS_BIRTH,TARGET
0,0.171468,0.699373,0.000000,-813,0.1667,-589.0,2,1,-8676,0
1,0.768808,0.199651,0.541385,-223,0.0417,-1409.0,2,1,-13583,0
2,0.626304,0.704880,0.000000,-6202,0.0000,-675.0,2,1,-13993,0
3,0.810618,0.724576,0.000000,365243,0.0000,-1588.0,1,0,-22425,0
4,0.331251,0.706310,0.592466,-2763,0.1667,0.0,2,1,-18839,0


In [14]:
X = credit_train_8.drop(['TARGET'],axis=1).values
y = credit_train_8[['TARGET']].values

In [15]:
print(len(X))
print(len(y))

246008
246008


**Ajustes de classes desbalanceadas**
- Aqui é utilizada a classe SMOTE para ajustar o balanceamento dos dados.
- https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html

In [ ]:
# smote = SMOTE()
# X_smote, y_smote = smote.fit_resample(X, y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=4)

**Aplicação do Pipeline**
- Aplicação do Pipeline apenas nos dados de treino.
- Cria uma função para o Grid Search CV.
- Adiciona os parâmetros de classificação do KNN.

In [16]:
def buildGridSearch(pipeline, 
                    param_grid):
    """ 
    Recebe como parâmetros um pipeline e parâmetros dos modelos e retorna o método
    GridSearchCV para selecionar os melhores parâmetros.
    """
    return GridSearchCV(pipeline, param_grid, scoring = 'roc_auc',refit=True, verbose = 1, cv=5, n_jobs = -1)

In [17]:
param_ =  {
    'classifier__n_neighbors': [1,3,6],
    'classifier__leaf_size': [20,40,1],
    'classifier__p': [1,2],
    'classifier__weights': ['uniform', 'distance'],
     'classifier__metric': ['minkowski','euclidean','manhattan']
    }




#não funciona com o Pipeline do sklearn
pipe = imbPipeline(steps=[
              ('num', SimpleImputer(strategy='median')),
              ('normalization',StandardScaler()),
              ('smote', SMOTE(random_state=5)),
              ('classifier', KNeighborsClassifier())

                ])

- Separa os dados de treino e teste.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

- Aplicação do Pipeline com o GridSearch

In [24]:
grid_ = buildGridSearch(pipe, param_).fit(X_train, y_train)

predict_ = grid_.predict_proba(X_test)[:,1]

roc_auc = roc_auc_score(y_test, predict_)

print("\n")
print(f"Métrica ROC: {roc_auc}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits


Métrica ROC: 0.6107123456899799


In [25]:
grid_.best_estimator_

Pipeline(steps=[('num', SimpleImputer(strategy='median')),
                ('normalization', StandardScaler()),
                ('smote', SMOTE(random_state=5)),
                ('classifier',
                 KNeighborsClassifier(leaf_size=20, n_neighbors=6, p=1))])

In [26]:
best_clf = grid_.best_estimator_

##Comentários
- Ao adicionar mais parâmetros ao modelo no pipeline o modelo ficou mais lento.
- Depois de aplicar os parâmetros o modelo KNN não melhorou significamente o resultado da métrica ROC-AUC.

**Ajuste nos dados de teste**
- Seleção das colunas e ajuste nos valores que possuem zero para a base de teste.

In [27]:
df_credit_test = df_credit_test[['SK_ID_CURR','EXT_SOURCE_3', 'EXT_SOURCE_2','EXT_SOURCE_1','DAYS_EMPLOYED',
                                  'FLOORSMAX_AVG','DAYS_LAST_PHONE_CHANGE','REGION_RATING_CLIENT','FLAG_EMP_PHONE','DAYS_BIRTH',]]

In [28]:
df_credit_test = df_credit_test.fillna(0)
df_credit_test.head()

,SK_ID_CURR,EXT_SOURCE_3,EXT_SOURCE_2,EXT_SOURCE_1,DAYS_EMPLOYED,FLOORSMAX_AVG,DAYS_LAST_PHONE_CHANGE,REGION_RATING_CLIENT,FLAG_EMP_PHONE,DAYS_BIRTH
0,149741,0.145543,0.182710,0.484936,-2125,0.0000,-2008.0,3,1,-21774
1,363290,0.680139,0.669053,0.624574,-2055,0.6250,-728.0,1,1,-18499
2,436006,0.173527,0.454279,0.000000,-4159,0.3333,-1273.0,2,1,-18084
3,377703,0.440058,0.688145,0.000000,-2532,0.0000,-984.0,2,1,-12319
4,188624,0.081726,0.426409,0.000000,-2516,0.1667,0.0,3,1,-10151


- Aplicação dos dados de teste

In [29]:
#aplica previsão ao dataframe
preds = best_clf.predict_proba(df_credit_test[['EXT_SOURCE_3', 'EXT_SOURCE_2','EXT_SOURCE_1','DAYS_EMPLOYED',
                                  'FLOORSMAX_AVG','DAYS_LAST_PHONE_CHANGE','REGION_RATING_CLIENT','FLAG_EMP_PHONE','DAYS_BIRTH']].values)[:,1]

In [30]:
df = pd.DataFrame(data = preds, columns=['PRED'], 
                  index=df_credit_test.index.copy())

df_final = pd.merge(df_credit_test[['SK_ID_CURR']], df, how = 'left', left_index = True, right_index = True)

print(df_final.head())

   SK_ID_CURR      PRED
0      149741  0.666667
1      363290  0.000000
2      436006  0.833333
3      377703  0.000000
4      188624  0.833333
